Universidad Galileo

Text Mining and Image Recognition

PAPD - Sección V

Sergio José Barrios Martínez

Carnet No. 19012765

## Laboratorio No. 2


***Instrucciones:*** 
A continuación verá una lista de ejercicios que debe completar para poder entregar el laboratorio #2. Al analizar, todos sus archivos deben estar contenidos en un archivo lab2-sucarnet.zip. Este archivo debe entregarlo en el link del GES. Por favor cree una carpeta para cada
ejercicio que usted realice.

In [4]:
import pandas as pd
import spacy
import spacy.cli
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pickle

### **Problema #1:**
Utilizando el código trabajado durante la sesión de laboratorio construya una función que le permita realizar la clasificación de un texto aleatorio proporcinado como argumento a dicha función. Su función debe utilzar los procedimientos y modelos de Machine Learning que se generaron en clase, si usted desea puede reescribir los procedimientos para volverlos más eficientes, el objetivo es que se pueda realizar la predicción de ham o spam para cualquier string dado.

### Función de Clasificación de Spam

In [207]:
def spam_classifier(corpus,modelo):
    
    nlp = spacy.load('es_core_news_sm')
    
    # Normalización
    newCorpus = []
    for doc in corpus:
        newCorpus.append(re.sub(r'[^a-zA-Z0-9\s{1}áéíóúü]','',doc).lower().strip().rstrip('\n').rstrip('\r\n'))
    corpus = newCorpus
    
    # Tokenización
    documents = []
    for doc in corpus:
        documents.append(nlp(doc))
    corpus = documents
        
    # Remoción de StopWords
    documents = []
    for doc in corpus:
        s = ""
        for token in doc:
            if(token.is_stop == False):
                s = s + token.text + " "
        documents.append(s.strip())
    corpus = documents
        
    # Stemming - Lematización
    documents = []
    for doc in corpus:
        documents.append(nlp(doc))
    corpus = documents

    documents = []
    for doc in corpus:
        s = ""
        for token in doc:
            s = s + token.lemma_ + " "
        documents.append(s.strip())
    corpus = documents
    
    # String de Elementos Unicos
    strCorpus = ""
    for palabra in corpus:
        strCorpus = strCorpus + palabra + " "
    strCorpus = strCorpus.strip().split(' ')
    setCorpus = set(strCorpus)
    
    # Creación del Dataframe para el Modelo
    corpusCols = list(setCorpus)
    corpusRows = range(0,len(corpus))
    outDf = pd.DataFrame(index=corpusRows, columns=corpusCols)
    outDf = outDf.fillna(0)
    
    # Cálculo de TF
    documents = []
    for doc in corpus:
        documents.append(nlp(doc))
    corpus = documents
    
    for index, doc in enumerate(corpus):
        bagOfWordsLen = len(doc)
        
        for word in doc:
            try:
                colIndex = list(outDf.columns).index(word.text)
                outDf.iloc[index,colIndex] = outDf.iloc[index,colIndex] + 1
            except:
                pass
      
        if bagOfWordsLen != 0:
            outDf.iloc[index,:] = outDf.iloc[index,:]/bagOfWordsLen
        else:
            outDf.iloc[index,:] = 0
    tfMatrix = outDf
    
    # Cálculo de IDF
    N = outDf.shape[0]
    valX = (N/outDf.astype(bool).sum(axis=0))
    corpusIDF = pd.Series(np.log(valX))
    
    # Corpus TF - IDF
    tfIdfCorpus = tfMatrix.mul(corpusIDF,axis=1)
    tfIdfCorpus = tfIdfCorpus.fillna(0)
    tfIdfCorpus
    
    
    # Features del modelo
    filename = 'features.sav'
    features = pickle.load(open(filename, 'rb'))
    
    # Creación del Dataframe para el Modelo
    observacionesCols = list(features)
    columnas = len(observacionesCols)
    observacionesRows = range(0,len(tfIdfCorpus))
    dataframeX = pd.DataFrame(0.0,index=observacionesRows, columns=observacionesCols)
    
    # Se actualiza el Dataframe de Predicción con los datos
    for fila in range(0,tfIdfCorpus.shape[0]):
        for caracteristica in tfIdfCorpus.columns:    
            i=0
            for columna in dataframeX.columns:            
                if caracteristica == columna:
                    dataframeX.loc[fila].at[caracteristica]=tfIdfCorpus.loc[fila].at[caracteristica]
                i+=1
    
    # Predicciones dependiendo del modelo
    if modelo == "SVM":
        filename = 'SVM.sav'
        svmc = pickle.load(open(filename, 'rb'))
        y_preds = svmc.predict(dataframeX)
    else:
        filename = 'random_forest.sav'
        rfc = pickle.load(open(filename, 'rb'))
        y_preds = rfc.predict(dataframeX)
           
    return y_preds    

### Prueba sobre muestra aleatoria Dataset Original

In [201]:
strSpamDB = 'es_spam.csv'
spamDB = pd.read_csv(strSpamDB, sep=',',names=['label','message'])
spamDB = spamDB.iloc[1:]
ham = spamDB[spamDB['label'] == 'ham']
spam = spamDB[spamDB['label'] == 'spam']
ham = ham.sample(2*spam.shape[0])
ham.shape, spam.shape
dataset = ham.append(spam,ignore_index=True)
dataset = dataset.sample(9)
dataset

,label,message
1345,ham,Jaja ... puedo ... Pero voy a cenar con mi pri...
1336,ham,"Hola, Según la petición de & lt; # & gt; Rs.5 ..."
1497,spam,Tenido sus teléfonos 11 meses o más? T R derec...
387,ham,Bastante tarde lar ... Ard 12 de todos modos y...
992,ham,Haciendo mi maestría. Cuando va a comprar un b...
1874,spam,Para ur oportunidad de ganar un dinero en efec...
489,ham,O simplemente hacer que 6times
1601,spam,T está suscrito a la mejor de servicios de con...
132,ham,"ok Ya, vikky vl c witin & lt; # & gt; minutos ..."


### Predicción con Random Forest

In [208]:
PruebaRandomForest=spam_classifier(dataset.message,"RandomForest")
PruebaRandomForest

array([0, 0, 1, 0, 0, 1, 0, 1, 0])

### Predicción con SVM

In [209]:
PruebaSVM=spam_classifier(dataset.message,"SVM")
PruebaSVM

array([0, 0, 1, 0, 0, 1, 0, 1, 0])

### Prueba con Texto Random 

In [244]:
texto1 = "Oferta de vacaciones de verano gratis"
texto2 = "Gane dinero sin esfuerzo"
texto3 = "Buenas tardes Ingeniero"
data = {'Texto':  [texto1,texto2,texto3]}
daf = pd.DataFrame (data, columns = ['Texto'])
TextoAleatorioSVM=spam_classifier(daf.Texto,"SVM")
TextoAleatorioSVM

array([1, 1, 0])